In [37]:
import numpy as np
import pandas as pd

In [38]:
resultsDF = pd.read_csv("../../evaluation_longTerm/results/result_raw", sep=",", header=0)
resultsDF.head()

,alg,group_id,user_id,metric,result
0,mf_30_1.0_avg_sim_group_2_AVG-F,0,3456,AR,2.734497
1,mf_30_1.0_avg_sim_group_2_AVG-F,0,3456,nDCG,0.851243
2,mf_30_1.0_avg_sim_group_2_AVG-F,0,5488,AR,2.735564
3,mf_30_1.0_avg_sim_group_2_AVG-F,0,5488,nDCG,0.784615
4,mf_30_1.0_avg_sim_group_2_AVG-F,1,4201,AR,1.704677


In [39]:
resultsDF.shape

(2000160, 5)

In [40]:
groupDF = resultsDF.groupby(["alg","group_id","user_id","metric"]).agg("mean")[["result"]]
groupDF.reset_index(inplace=True)
groupDF.head()

,alg,group_id,user_id,metric,result
0,mf_30_1.0_avg_div_group_2_AVG-F,0,3148,AR,1.430196
1,mf_30_1.0_avg_div_group_2_AVG-F,0,3148,nDCG,0.662144
2,mf_30_1.0_avg_div_group_2_AVG-F,0,4075,AR,1.910713
3,mf_30_1.0_avg_div_group_2_AVG-F,0,4075,nDCG,0.669084
4,mf_30_1.0_avg_div_group_2_AVG-F,1,332,AR,2.642345


In [41]:
groupDF.shape

(400032, 5)

In [42]:
groupDF.to_csv("../../evaluation_longTerm/results/res_group_folds.csv", sep=",", header=True)

In [43]:
from pathlib import Path
import sys, os
from typing import Dict, List, NamedTuple, Tuple
import numpy as np
from collections import defaultdict
import time


In [44]:
def compute_metrics(alg_name, groups, results):
    # test_data are triplets: user_id, item_id, and rating
    #LP: test data is matrix user_id x item_id !!!!!! a ja si rikal, jakto ze ti to prirazeni funguje...

    
    
    avg_rating = []
    min_rating = []
    minmax_rating = []
    std_rating = []
    
    avg_nDCG_rating = []
    min_nDCG_rating = []
    minmax_nDCG_rating = []
    std_nDCG_rating = []
        
    for gid in groups:
        gid_res_ndcg = results.loc[(results.group_id == gid)&(results.metric == "nDCG")]
        gid_res_AR = results.loc[(results.group_id == gid)&(results.metric == "AR")] 
        
        #print(gid_res_ndcg)
        #print(gid_res_AR)
        
        group_users_mean_ratings = [i for i in gid_res_AR.result]
        group_users_ndcg_ratings = [i for i in gid_res_ndcg.result]
        
        #print(group_users_mean_ratings)
        if len(group_users_mean_ratings) > 0:
            avg_rating.append(np.average(group_users_mean_ratings)) 
            min = np.min(group_users_mean_ratings)
            min_rating.append(min) 
            max = np.max(group_users_mean_ratings)
            minmax_rating.append(0.0 if max == 0.0 else min/max)
            std_rating.append(np.std(group_users_mean_ratings)) 

            avg_nDCG_rating.append(np.average(group_users_ndcg_ratings)) 
            min = np.min(group_users_ndcg_ratings)
            min_nDCG_rating.append(min) 
            max = np.max(group_users_ndcg_ratings)
            minmax_nDCG_rating.append(0.0 if max == 0.0 else min/max)
            std_nDCG_rating.append(np.std(group_users_ndcg_ratings)) 

    print(len(avg_rating))    
    results = pd.DataFrame({
       "AR_avg": avg_rating ,
        "AR_min": min_rating ,
        "AR_min/max": minmax_rating ,
        #"AR_std": std_rating ,
        
        "nDCG_avg": avg_nDCG_rating ,
        "nDCG_min": min_nDCG_rating ,
        "nDCG_min/max": minmax_nDCG_rating #,
        #"nDCG_std": std_nDCG_rating 
    })
      
    return results

In [45]:
algs = resultsDF.alg.unique()
groups = resultsDF.group_id.unique()
metrics = resultsDF.metric.unique()

In [35]:
from scipy import stats
#Comparison for size 8
def calculate_significance(results, grouptype, size, algs):
    print("")
    print(grouptype,size)

    otherAlgs = [a for a in algs if ( grouptype+"_group_"+size in a )]

    for count1, alg1 in enumerate(otherAlgs):
        baseDF = results[alg1]
        
        for count2,alg in enumerate(otherAlgs):
            if count1 != count2:
                otherDF = results[alg]
                res = {}
                for col in otherDF.columns:
                    mean_first = np.mean(baseDF.loc[:,col])
                    mean_second = np.mean(otherDF.loc[:,col])
                    mean_diff = np.mean(baseDF.loc[:,col] - otherDF.loc[:,col])
                    stat,pval = stats.ttest_rel(baseDF.loc[:,col], otherDF.loc[:,col])
                    if pval > 0.0001:
                        print("pval_excess",col)
                    res[col] = (mean_first,mean_second,mean_diff, pval)

                alg1 = alg1.replace("mf_230_1.0_avg_","")
                alg1 = alg1.replace(grouptype+"_group_"+size+"_","")
                alg = alg.replace("mf_230_1.0_avg_","")
                alg = alg.replace(grouptype+"_group_"+size+"_","")
                print(alg1+" - "+ alg)
                print(pd.DataFrame(res, index=["first","second","diff","pval"]))
                print("")

In [47]:
results_all = {}
algs_sign = []
whitelist = [ 'AVG-F', 'FuzzyDHondtDirectOptimize_1','FuzzyDHondtDirectOptimize-F',  'FuzzyDHondt-F']
for grouptype in ["sim","div"]:
    for size in ["2","3","4","8"]:
        resultsDF = groupDF.loc[groupDF.alg.str.contains(grouptype+"_group_"+size)]        
        algs = resultsDF.alg.unique()   
        print(algs)
        metrics = resultsDF.metric.unique()
        groups = resultsDF.group_id.unique()        
        results = {}
        for a in algs:
            for item in whitelist:
                if item in a:
                    #print(a)
                    algs_sign.append(a)
                    resPerAlg = resultsDF.loc[resultsDF.alg == a]            
                    results[a] = compute_metrics(a, groups, resPerAlg)
        calculate_significance(results, grouptype, size, algs_sign)

['mf_30_1.0_avg_sim_group_2_AVG-F'
 'mf_30_1.0_avg_sim_group_2_FuzzyDHondt-F'
 'mf_30_1.0_avg_sim_group_2_FuzzyDHondtDirectOptimize-F'
 'mf_30_1.0_avg_sim_group_2_FuzzyDHondtDirectOptimize_1.0']
942
942
942
942

sim 2
mf_30_1.0_avg_AVG-F - mf_30_1.0_avg_FuzzyDHondt-F
              AR_avg         AR_min     AR_min/max  nDCG_avg      nDCG_min  \
first   2.818147e+00   2.493314e+00   7.984939e-01  0.840096  8.110554e-01   
second  2.807640e+00   2.567177e+00   8.438928e-01  0.840311  8.261543e-01   
diff    1.050708e-02  -7.386321e-02  -4.539894e-02 -0.000215 -1.509891e-02   
pval    2.286508e-81  2.099624e-161  6.079262e-179  0.000002  7.617752e-93   

        nDCG_min/max  
first   9.337529e-01  
second  9.669690e-01  
diff   -3.321608e-02  
pval    2.448799e-92  

mf_30_1.0_avg_AVG-F - mf_30_1.0_avg_FuzzyDHondtDirectOptimize-F
               AR_avg         AR_min     AR_min/max       nDCG_avg  \
first    2.818147e+00   2.493314e+00   7.984939e-01   8.400963e-01   
second   2.721032e+00

['mf_30_1.0_avg_sim_group_4_AVG-F'
 'mf_30_1.0_avg_sim_group_4_FuzzyDHondt-F'
 'mf_30_1.0_avg_sim_group_4_FuzzyDHondtDirectOptimize-F'
 'mf_30_1.0_avg_sim_group_4_FuzzyDHondtDirectOptimize_1.0']
937
937
937
937

sim 4
mf_30_1.0_avg_AVG-F - mf_30_1.0_avg_FuzzyDHondt-F
               AR_avg         AR_min  AR_min/max      nDCG_avg      nDCG_min  \
first    2.381101e+00   1.881037e+00    0.659362  6.888506e-01  6.271628e-01   
second   2.367967e+00   2.002010e+00    0.736580  6.882696e-01  6.419599e-01   
diff     1.313406e-02  -1.209731e-01   -0.077218  5.810591e-04 -1.479713e-02   
pval    9.443628e-172  1.902805e-252    0.000000  5.173734e-10  7.789915e-45   

        nDCG_min/max  
first   8.422331e-01  
second  8.801194e-01  
diff   -3.788625e-02  
pval    1.937922e-66  

mf_30_1.0_avg_AVG-F - mf_30_1.0_avg_FuzzyDHondtDirectOptimize-F
          AR_avg         AR_min  AR_min/max  nDCG_avg      nDCG_min  \
first   2.381101   1.881037e+00    0.659362  0.688851  6.271628e-01   
second  2

['mf_30_1.0_avg_div_group_2_AVG-F'
 'mf_30_1.0_avg_div_group_2_FuzzyDHondt-F'
 'mf_30_1.0_avg_div_group_2_FuzzyDHondtDirectOptimize-F'
 'mf_30_1.0_avg_div_group_2_FuzzyDHondtDirectOptimize_1.0']
1000
1000
1000
1000

div 2
mf_30_1.0_avg_AVG-F - mf_30_1.0_avg_FuzzyDHondt-F
              AR_avg         AR_min     AR_min/max      nDCG_avg  \
first   2.236545e+00   1.654794e+00   6.137824e-01  7.266085e-01   
second  2.199876e+00   1.852722e+00   7.388689e-01  7.360489e-01   
diff    3.666921e-02  -1.979279e-01  -1.250866e-01 -9.440446e-03   
pval    4.079292e-83  6.404074e-265  2.987590e-306  2.118597e-96   

            nDCG_min  nDCG_min/max  
first   6.719816e-01  8.618559e-01  
second  7.098080e-01  9.317903e-01  
diff   -3.782637e-02 -6.993442e-02  
pval    5.411003e-96  1.735612e-79  

mf_30_1.0_avg_AVG-F - mf_30_1.0_avg_FuzzyDHondtDirectOptimize-F
               AR_avg         AR_min  AR_min/max      nDCG_avg       nDCG_min  \
first    2.236545e+00   1.654794e+00    0.613782  7.2660

['mf_30_1.0_avg_div_group_4_AVG-F'
 'mf_30_1.0_avg_div_group_4_FuzzyDHondt-F'
 'mf_30_1.0_avg_div_group_4_FuzzyDHondtDirectOptimize-F'
 'mf_30_1.0_avg_div_group_4_FuzzyDHondtDirectOptimize_1.0']
1000
1000
1000
1000

div 4
mf_30_1.0_avg_AVG-F - mf_30_1.0_avg_FuzzyDHondt-F
               AR_avg    AR_min  AR_min/max      nDCG_avg       nDCG_min  \
first    1.842423e+00  1.079871    0.431684  5.886776e-01   4.642851e-01   
second   1.802203e+00  1.289732    0.568744  5.919124e-01   5.266934e-01   
diff     4.021946e-02 -0.209861   -0.137060 -3.234735e-03  -6.240825e-02   
pval    5.127268e-161  0.000000    0.000000  3.971395e-42  1.251073e-137   

         nDCG_min/max  
first    6.785817e-01  
second   8.063202e-01  
diff    -1.277385e-01  
pval    6.765464e-149  

mf_30_1.0_avg_AVG-F - mf_30_1.0_avg_FuzzyDHondtDirectOptimize-F
          AR_avg    AR_min  AR_min/max       nDCG_avg      nDCG_min  \
first   1.842423  1.079871    0.431684   5.886776e-01  4.642851e-01   
second  1.677643  1.